# Kernal Used: Python 2

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pgmpy.estimators import HillClimbSearch,BicScore,BayesianEstimator
from pgmpy.models import BayesianModel
from matplotlib import pyplot as plt
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from sklearn.ensemble import RandomForestClassifier

#import BNfinder as bnf

In [2]:
df = pd.read_csv("dataAll(2).csv")
df = df.drop("Unnamed: 0", axis = 1)
df = df.dropna()
print df.shape
df.head()

(1880, 10)


,zipcode,%Foreign,%Uninsured,%vehicle,%foodStamp,%poverty,urban,%TeenBirthRate,%PrematureBirth,%LowBirthWeight
0,10001,16.230927,6.313218,77.510070,51.290074,17.5,1,5.7,10.5,11.0
1,10002,16.301731,7.873643,76.876850,42.674401,27.8,1,17.7,8.9,6.6
2,10003,12.463227,5.066324,77.210118,48.017334,9.9,1,0.6,9.7,8.7
3,10004,15.177398,6.241787,76.513672,52.299606,4.8,1,0.0,8.7,6.7
4,10005,16.211251,5.774971,85.056784,49.311137,12.1,1,0.0,10.7,6.0


In [3]:
df.columns

Index([u'zipcode', u'%Foreign', u'%Uninsured', u'%vehicle', u'%foodStamp',
       u'%poverty', u'urban', u'%TeenBirthRate', u'%PrematureBirth',
       u'%LowBirthWeight'],
      dtype='object')

## Premature Birth with Teen Birth Rate

In [13]:
dfunpre = df[['%Uninsured','%vehicle','%foodStamp','%poverty',
          '%Foreign','%TeenBirthRate','urban','%PrematureBirth']]
print dfunpre.shape
dfunpre.head()

(1880, 8)


,%Uninsured,%vehicle,%foodStamp,%poverty,%Foreign,%TeenBirthRate,urban,%PrematureBirth
0,6.313218,77.510070,51.290074,17.5,16.230927,5.7,1,10.5
1,7.873643,76.876850,42.674401,27.8,16.301731,17.7,1,8.9
2,5.066324,77.210118,48.017334,9.9,12.463227,0.6,1,9.7
3,6.241787,76.513672,52.299606,4.8,15.177398,0.0,1,8.7
4,5.774971,85.056784,49.311137,12.1,16.211251,0.0,1,10.7


In [14]:
dfunpre.dtypes

%Uninsured         float64
%vehicle           float64
%foodStamp         float64
%poverty           float64
%Foreign           float64
%TeenBirthRate     float64
urban                int64
%PrematureBirth    float64
dtype: object

In [15]:
for i in dfunpre:
    dfunpre[i] = pd.qcut(dfunpre[i].rank(method='first'),4,labels=False)
print dfunpre.head()

/Users/emilypadvorac/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


   %Uninsured  %vehicle  %foodStamp  %poverty  %Foreign  %TeenBirthRate  \
0           2         3           3         3         3               1   
1           2         3           3         3         3               2   
2           1         3           3         1         3               0   
3           2         3           3         0         3               0   
4           1         3           3         2         3               0   

   urban  %PrematureBirth  
0      1                2  
1      1                1  
2      1                1  
3      1                1  
4      1                2  


In [16]:
dfunpre.dtypes

%Uninsured         int64
%vehicle           int64
%foodStamp         int64
%poverty           int64
%Foreign           int64
%TeenBirthRate     int64
urban              int64
%PrematureBirth    int64
dtype: object

In [17]:
dfunpre['%PrematureBirth'].value_counts()

3    470
2    470
1    470
0    470
Name: %PrematureBirth, dtype: int64

In [18]:
train,test=train_test_split(dfunpre,random_state=999,test_size=0.4)

In [19]:
hc = HillClimbSearch(train, scoring_method=BicScore(train))
best_model = hc.estimate()
print(best_model.edges())

[('urban', '%foodStamp'), ('%Foreign', 'urban'), ('%TeenBirthRate', '%poverty'), ('%poverty', '%Uninsured'), ('%poverty', '%vehicle'), ('%vehicle', '%PrematureBirth'), ('%vehicle', '%Foreign')]


In [20]:
best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of urban:
+----------+---------------------+---------------------+---------------------+----------------------+
| %Foreign | %Foreign(0)         | %Foreign(1)         | %Foreign(2)         | %Foreign(3)          |
+----------+---------------------+---------------------+---------------------+----------------------+
| urban(0) | 0.25092250922509224 | 0.24475524475524477 | 0.30952380952380953 | 0.20819112627986347  |
+----------+---------------------+---------------------+---------------------+----------------------+
| urban(1) | 0.4575645756457565  | 0.32867132867132864 | 0.16666666666666666 | 0.034129692832764506 |
+----------+---------------------+---------------------+---------------------+----------------------+
| urban(2) | 0.04428044280442804 | 0.07692307692307693 | 0.23809523809523808 | 0.6552901023890785   |
+----------+---------------------+---------------------+---------------------+----------------------+
| urban(3) | 0.24723247232472326 | 0.34965034965034963 | 0.285714285

In [21]:
predicted_test = best_model.predict(test.loc[:,"%Uninsured":"urban"])
print "Out of sample:",(test.loc[:,"%PrematureBirth"]==predicted_test["%PrematureBirth"]).mean()

Out of sample: 0.2925531914893617


In [22]:
predicted_train = best_model.predict(train.loc[:,"%Uninsured":"urban"]-1)
print "In sample:",(train.loc[:,'%PrematureBirth']==predicted_train['%PrematureBirth']).mean()
predicted_test = best_model.predict(test.loc[:,"%Uninsured":"urban"]-1)
print "Out of sample:",(test.loc[:,'%PrematureBirth']==predicted_test['%PrematureBirth']).mean()

In sample: 0.2632978723404255
Out of sample: 0.28191489361702127


## Low Birth Weight with Teen Birth Rate

In [32]:
dflbpren = df[['%Uninsured','%vehicle','%foodStamp','%poverty',
          '%Foreign','%TeenBirthRate','urban','%LowBirthWeight']]
print dflbpren.shape
dflbpren.head()

(1880, 8)


,%Uninsured,%vehicle,%foodStamp,%poverty,%Foreign,%TeenBirthRate,urban,%LowBirthWeight
0,6.313218,77.510070,51.290074,17.5,16.230927,5.7,1,11.0
1,7.873643,76.876850,42.674401,27.8,16.301731,17.7,1,6.6
2,5.066324,77.210118,48.017334,9.9,12.463227,0.6,1,8.7
3,6.241787,76.513672,52.299606,4.8,15.177398,0.0,1,6.7
4,5.774971,85.056784,49.311137,12.1,16.211251,0.0,1,6.0


In [33]:
for i in dflbpren:
    dflbpren[i] = pd.qcut(dflbpren[i].rank(method='first'),4,labels=False)
print dflbpren.head()

/Users/emilypadvorac/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


   %Uninsured  %vehicle  %foodStamp  %poverty  %Foreign  %TeenBirthRate  \
0           2         3           3         3         3               1   
1           2         3           3         3         3               2   
2           1         3           3         1         3               0   
3           2         3           3         0         3               0   
4           1         3           3         2         3               0   

   urban  %LowBirthWeight  
0      1                3  
1      1                1  
2      1                2  
3      1                1  
4      1                1  


In [34]:
dflbpren['%LowBirthWeight'].value_counts()

3    470
2    470
1    470
0    470
Name: %LowBirthWeight, dtype: int64

In [35]:
train,test=train_test_split(dflbpren,random_state=999,test_size=0.4)

In [36]:
hc = HillClimbSearch(train, scoring_method=BicScore(train))
best_model = hc.estimate()
print(best_model.edges())

[('urban', '%foodStamp'), ('%Foreign', 'urban'), ('%TeenBirthRate', '%poverty'), ('%poverty', '%Uninsured'), ('%poverty', '%vehicle'), ('%vehicle', '%LowBirthWeight'), ('%vehicle', '%Foreign')]


In [37]:
best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of urban:
+----------+---------------------+---------------------+---------------------+----------------------+
| %Foreign | %Foreign(0)         | %Foreign(1)         | %Foreign(2)         | %Foreign(3)          |
+----------+---------------------+---------------------+---------------------+----------------------+
| urban(0) | 0.25092250922509224 | 0.24475524475524477 | 0.30952380952380953 | 0.20819112627986347  |
+----------+---------------------+---------------------+---------------------+----------------------+
| urban(1) | 0.4575645756457565  | 0.32867132867132864 | 0.16666666666666666 | 0.034129692832764506 |
+----------+---------------------+---------------------+---------------------+----------------------+
| urban(2) | 0.04428044280442804 | 0.07692307692307693 | 0.23809523809523808 | 0.6552901023890785   |
+----------+---------------------+---------------------+---------------------+----------------------+
| urban(3) | 0.24723247232472326 | 0.34965034965034963 | 0.285714285

In [38]:
predicted_test = best_model.predict(test.loc[:,"%Uninsured":"urban"])
print "Out of sample:",(test.loc[:,'%LowBirthWeight']==predicted_test['%LowBirthWeight']).mean()

Out of sample: 0.3071808510638298


In [39]:
predicted_train = best_model.predict(train.loc[:,"%Uninsured":"urban"]-1)
print "In sample:",(train.loc[:,'%LowBirthWeight']==predicted_train['%LowBirthWeight']).mean()
predicted_test = best_model.predict(test.loc[:,"%Uninsured":"urban"]-1)
print "Out of sample:",(test.loc[:,'%LowBirthWeight']==predicted_test['%LowBirthWeight']).mean()

In sample: 0.25443262411347517
Out of sample: 0.2912234042553192
